<a href="https://colab.research.google.com/github/bellatchen/Assignments/blob/main/Abalone_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bella Chen

Tasks:

3) Use column transformers to transform the appropriate columns in the appropriate ways.

For the Column Transformations:

    a) Select the categorical columns and the numerical columns

    b) Use a OneHotEncoder to encode the categorical columns

    c) Use a StandardScaler to scale the numeric columns

    d) Use a ColumnTransformer to match the transformation to the type of column

    e) Transform the data and display the resulting Numpy array.
  

In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [14]:
# mounting Data
abalone = "/content/drive/MyDrive/Coding Dojo/Data Sets/abalone.data"
abalone_df_original = pd.read_csv(abalone)
abalone_df_original
abalone_df_1 = abalone_df_original.copy(deep= True)
print(abalone_df_1)

     Sex  Length  Diameter  Height  Whole Weight  Shucked Weight  \
0      M   0.455     0.365   0.095        0.5140          0.2245   
1      M   0.350     0.265   0.090        0.2255          0.0995   
2      F   0.530     0.420   0.135        0.6770          0.2565   
3      M   0.440     0.365   0.125        0.5160          0.2155   
4      I   0.330     0.255   0.080        0.2050          0.0895   
...   ..     ...       ...     ...           ...             ...   
4172   F   0.565     0.450   0.165        0.8870          0.3700   
4173   M   0.590     0.440   0.135        0.9660          0.4390   
4174   M   0.600     0.475   0.205        1.1760          0.5255   
4175   F   0.625     0.485   0.150        1.0945          0.5310   
4176   M   0.710     0.555   0.195        1.9485          0.9455   

      Viscera Weight  Shell Weight  Rings  
0             0.1010        0.1500     15  
1             0.0485        0.0700      7  
2             0.1415        0.2100      9  
3      

In [15]:
#Checking for Duplicates
abalone_df_1.duplicated().sum()

0

In [16]:
#check for missing data
abalone_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole Weight    4177 non-null   float64
 5   Shucked Weight  4177 non-null   float64
 6   Viscera Weight  4177 non-null   float64
 7   Shell Weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


In [17]:
#checking values and counts
abalone_df_1["Sex"].value_counts()
#everything is uniform

M    1528
I    1342
F    1307
Name: Sex, dtype: int64

In [19]:
# Separate data into features matrix (X) and target vector (y)
#defining X and y; predicting charges
X = abalone_df_1.drop(columns=["Rings"])
y = abalone_df_1["Rings"]
# Train/test split the data. Please use the random number 42 for consistency.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [20]:
 #verifying split
 len(X_train)

3132

In [21]:
# One Hot Encode any nominal features 
#make categorical selector
cat_selector = make_column_selector(dtype_include='object')
cat_selector(X_train)

# create a subset of data for only categorical columns
train_cat_data = X_train[cat_selector(X_train)]
test_cat_data = X_test[cat_selector(X_test)]

#instantiate one hot encoder
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

#fit the OneHotEncoder on the training data
ohe_encoder.fit(train_cat_data)

#transform both the training and the testing data
train_ohe = ohe_encoder.transform(train_cat_data)
test_ohe = ohe_encoder.transform(test_cat_data)

#convert to dataframe, extract new column names from encoder
#set prefixes to original column names
ohe_column_names = ohe_encoder.get_feature_names_out(train_cat_data.columns)
train_ohe = pd.DataFrame(train_ohe, columns=ohe_column_names)
test_ohe = pd.DataFrame(test_ohe, columns=ohe_column_names)

# create a numeric selector
num_selector = make_column_selector(dtype_include='number')

# isolate the numeric columns
train_nums = X_train[num_selector(X_train)].reset_index(drop=True)
test_nums = X_test[num_selector(X_test)].reset_index(drop=True)

# re-combine the train and test sets on axis 1 (columns)
X_train_processed = pd.concat([train_nums, train_ohe], axis=1)
X_test_processed = pd.concat([test_nums, test_ohe], axis=1)
X_train_processed


,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_F,Sex_I,Sex_M
0,0.615,0.455,0.135,1.0590,0.4735,0.2630,0.274,1.0,0.0,0.0
1,0.515,0.395,0.140,0.6860,0.2810,0.1255,0.220,1.0,0.0,0.0
2,0.660,0.530,0.175,1.5830,0.7395,0.3505,0.405,0.0,0.0,1.0
3,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,0.0,0.0,1.0
4,0.495,0.400,0.155,0.8085,0.2345,0.1155,0.350,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3127,0.490,0.400,0.115,0.5690,0.2560,0.1325,0.145,1.0,0.0,0.0
3128,0.670,0.550,0.190,1.3905,0.5425,0.3035,0.400,1.0,0.0,0.0
3129,0.510,0.395,0.125,0.5805,0.2440,0.1335,0.188,0.0,0.0,1.0
3130,0.575,0.465,0.120,1.0535,0.5160,0.2185,0.235,0.0,0.0,1.0


In [29]:
# Scale any numeric features
X_train.describe().round(2)
# instantiate scaler
scaler = StandardScaler()
# fit scaler on training data
scaler.fit(X_train_processed)
# transform training data
X_train_processed_scaled = scaler.transform(X_train_processed)
X_test_processed_scaled = scaler.transform(X_test_processed)
X_train_processed_scaled[:5]


array([[ 7.49290761e-01,  4.64225837e-01, -1.18869233e-01,
         4.57446573e-01,  4.99097874e-01,  7.43972878e-01,
         2.41134902e-01,  1.46984489e+00, -6.80846093e-01,
        -7.61199786e-01],
       [-9.02537114e-02, -1.44654423e-01, -1.64680298e-03,
        -3.01654922e-01, -3.64269333e-01, -5.14039773e-01,
        -1.45837777e-01,  1.46984489e+00, -6.80846093e-01,
        -7.61199786e-01],
       [ 1.12708577e+00,  1.22532616e+00,  8.18910208e-01,
         1.52385189e+00,  1.69211438e+00,  1.54452638e+00,
         1.17990196e+00, -6.80343897e-01, -6.80846093e-01,
         1.31371556e+00],
       [-5.93980395e-01, -4.49094554e-01, -1.05664867e+00,
        -6.51696363e-01, -6.17673214e-01, -7.38194754e-01,
        -6.47469029e-01, -6.80343897e-01, -6.80846093e-01,
         1.31371556e+00],
       [-2.58162606e-01, -9.39144017e-02,  3.50020487e-01,
        -5.23521522e-02, -5.72822969e-01, -6.05531602e-01,
         7.85763118e-01, -6.80343897e-01, -6.80846093e-01,
         1.